In [1]:
import csv
import pandas as pd
import numpy as np
import us

In [2]:
df_2021 = pd.read_csv("PERM_Disclosure_Data_FY2021.csv", low_memory=False)
df_2022 = pd.read_csv("PERM_Disclosure_Data_FY2022_Q1.csv", low_memory=False)

In [3]:
def normalize(data):
    res = data.replace(',', '')
    res = res.replace('.', '')
    res = res.upper()
    return res

def clean_up(data):
    res = data.replace('"', '')
    res = res.replace('\'', '')
    res = res.replace(',', '')
    res = res.replace('  ', '')
    res = res.upper()
    return res

def state_to_code(state_name):
    if state_name == None:
        return state_name
    state_name = ''.join([i for i in state_name if not i.isdigit()])
    state = us.states.lookup(state_name)
    if state == None:
        return state_name
    return state.abbr

In [4]:
df_2021['EMPLOYER_NAME'] = df_2021['EMPLOYER_NAME'].apply(normalize)
df_2022['EMPLOYER_NAME'] = df_2022['EMPLOYER_NAME'].apply(normalize)
companies = pd.read_csv("companies.csv", low_memory=False)
companies

,COMPANY_ID,NAME,ADDRESS_1,ADDRESS_2,CITY,STATE,COUNTRY,ZIP_CODE
0,0,400 CAPITAL MANAGEMENT LLC,510 MADISON AVE,17TH FLOOR,NEW YORK,NY,UNITED STATES OF AMERICA,10025
1,1,ABN AMRO CLEARING CHICAGO LLC,175 W JACKSON BOULEVARD,SUITE 2050,CHICAGO,IL,UNITED STATES OF AMERICA,60604
2,2,ACHELOUS PARTNERS,800 3RD AVENUE,21ST FLOOR,NEW YORK,NY,UNITED STATES OF AMERICA,10022
3,3,AES US SERVICES LLC,4300 WILSON BLVD,NaN,ARLINGTON,VA,UNITED STATES OF AMERICA,22203
4,4,AKUNA CAPITAL LLC,333 S. WABASH AVE.,SUITE 2600,CHICAGO,IL,UNITED STATES OF AMERICA,60604
...,...,...,...,...,...,...,...,...
287,287,WEST COAST FLORIDA ENTERPRISES INC,8090 SUPPLY DRIVE,STE 100,FT. MYERS,FL,UNITED STATES OF AMERICA,33912
288,288,WILLIAM BLAIR COMPANY LLC,150 N. RIVERSIDE PLAZA,NaN,CHICAGO,IL,UNITED STATES OF AMERICA,60606
289,289,WORLDQUANT LLC,1700 EAST PUTNAM AVENUE,NaN,OLD GREENWICH,CT,UNITED STATES OF AMERICA,6870
290,290,WORLDQUANT PREDICTIVE TECHNOLOGIES,575 5TH AVE,NaN,NEW YORK CITY,NY,UNITED STATES OF AMERICA,10017


In [5]:
companies2 = pd.read_csv("companies_NSCC-MPID_fuzzymatched.csv", low_memory=False)
companies2

,COMPANY_ID,NAME,ADDRESS_1,ADDRESS_2,CITY,STATE,COUNTRY,ZIP_CODE
0,292,C6 CAPITAL SECURITIES LLC,600 Lexington Ave,32nd floor,New York,NY,UNITED STATES OF AMERICA,10022 ...
1,293,NATIONAL ALLIANCE SECURITIES CORPORATION,505 Park Ave,# 502,New York,NY,UNITED STATES OF AMERICA,10022
2,294,RIM SECURITIES LLC,400 Park Ave,NaN,New York,NY,UNITED STATES OF AMERICA,10022 ...
3,295,KMS FINANCIAL SERVICES,2001 6th Ave,#2801,Seattle,WA,UNITED STATES OF AMERICA,98121
4,296,VSR FINANCIAL SERVICES,8620 W 110TH STREET,200,OVERLAND PARK,KS,UNITED STATES OF AMERICA,66210-9651
5,297,ICP SECURITIES LLC,360 Madison Ave,NaN,New York,NY,UNITED STATES OF AMERICA,10017 ...


In [6]:
companies3 = pd.read_csv("companies_H1B_fuzzymatched.csv", low_memory=False)
companies3

,COMPANY_ID,NAME,CITY,STATE
0,298,ROSEWOOD INVESTMENT CORPORATION,NEW YORK,NY
1,299,SEMPER CAPITAL MANAGEMENT LP,NEW YORK,NY
2,300,VOYA INVESTMENT MANAGEMENT LLC,ATLANTA,GA
3,301,HQ CAPITAL PRIVATE EQUITY LLC,NEW YORK,NY
4,302,WATERFALL ASSET MANAGEMENT LLC,NEW YORK,NY
...,...,...,...,...
254,552,GATEWOOD WEALTH SOLUTIONS,ST. LOUIS,MO
255,553,B RILEY FBR INC,NEW YORK,NY
256,554,ALTISOURCE SOLUTIONS INC,PLANO,TX
257,555,PEOPLE'S UNITED ADVISORS INC,NEW YORK,NY


In [7]:
company_roles = df_2021[df_2021['EMPLOYER_NAME'].isin(companies['NAME']) | df_2021['EMPLOYER_NAME'].isin(companies2['NAME']) | df_2021['EMPLOYER_NAME'].isin(companies3['NAME'])].copy()
company_roles['WORKSITE_CITY'] = company_roles['WORKSITE_CITY'].str.upper()
company_roles['WORKSITE_STATE'] = company_roles['WORKSITE_STATE'].str.upper()
company_roles['JOB_TITLE'] = company_roles['JOB_TITLE'].str.upper()
company_roles['JOB_TITLE'] = company_roles['JOB_TITLE'].apply(clean_up)
roles = company_roles['JOB_TITLE'].unique()
roles

array(['AEM ADMINISTRATOR', 'MANAGER', 'QUANTITATIVE SOFTWARE ENGINEER',
       ..., 'DEVELOPMENT MANAGER', 'SENIOR SOFTWARE DEVELOPER – RESEARCH',
       'CROSS ASSET FINANCIAL ENGINEER'], dtype=object)

In [8]:
company_roles2 = df_2022[df_2022['EMPLOYER_NAME'].isin(companies['NAME']) | df_2022['EMPLOYER_NAME'].isin(companies2['NAME']) | df_2022['EMPLOYER_NAME'].isin(companies3['NAME'])].copy()
company_roles2['WORKSITE_CITY'] = company_roles2['WORKSITE_CITY'].str.upper()
company_roles2['WORKSITE_STATE'] = company_roles2['WORKSITE_STATE'].str.upper()
company_roles2['JOB_TITLE'] = company_roles2['JOB_TITLE'].str.upper()
company_roles2['JOB_TITLE'] = company_roles2['JOB_TITLE'].apply(clean_up)
roles2 = company_roles2['JOB_TITLE'].unique()
roles2

array(['QUANTITATIVE TRADER', 'DIRECTOR FINANCE',
       'DIRECTOR / BA LEAD MKTS (MGR)',
       'ASSOCIATE GLOBAL FUND AND ACCOUNTING SERVICES',
       'EXECUTIVE DIRECTOR CREDIT RISK',
       'ASSOCIATE MIDWEST ADVISORY GROUP',
       'VICE PRESIDENT SALES AND TRADING',
       'ASSOCIATE DATA & AI SERVICES - REFERENCE & MARKET DATA',
       'ASSOCIATE BUSINESS OPERATIONS DATA SERVICES INTEGRATION',
       'DIRECTOR; INVESTMENT BANKER IV',
       'QUANTITATIVE ANALYST-QUANTITATIVE RISK',
       'GP&P SENIOR MANAGER (VP)', 'VICE PRESIDENT GLOBAL ALLOCATION',
       'RCA (RISK/COMPLIANCE/AUDIT) PROFESSIONAL',
       'CHIEF ENGINEER (MULTIPLE POSITIONS)',
       'SOFTWARE DEVELOPER (MULTIPLE POSITIONS)',
       'CHIEF ENGINER (MULTIPLE POSITIONS)',
       'IT ADV PI - TECH CONS - TECH TRANS - TECH OPS - MANAGER',
       'APPLICATION SUPPORT', 'SOFTWARE DEVELOPER',
       'SOFTWARE ENGINEERING',
       'ASSISTANT VICE PRESIDENT / PROGRAMMER MKTS',
       'VICE PRESIDENT / PROGRAMMER PROF 

In [34]:
all_roles = np.unique(np.concatenate((roles, roles2), axis=None))
all_role_df = pd.DataFrame(all_roles, columns = ['NAME'])
all_role_df = all_role_df.reset_index(drop=True)
all_role_df.to_csv('roles.csv')

In [11]:
roles = pd.read_csv("roles_final.csv", low_memory=False)
roles

,ROLE_ID,NAME
0,0,.NET & FRONT-END DEVELOPER
1,1,.NET APPLICATION DEVELOPER- TEST AUTOMATION
2,2,.NET AUTOMATION DEVELOPER
3,3,.NET DEVELOPER
4,4,.NET DEVELOPER ASSISTANT VICE PRESIDENT
...,...,...
22645,22645,WORLD TRADER AND GLOBALIZATION SENIOR REPORTER
22646,22646,WQ INFRASTRUCTURE ALIGNED SYSTEMS ENGINEER
22647,22647,XVA QUANT- VICE PRESIDENT
22648,22648,YIELD BOOK ANALYST


In [14]:
cr = pd.DataFrame(columns = ['COMPANY_ID', 'ROLE_ID'])

for index, row in companies.iterrows():
    temp = company_roles.loc[company_roles['EMPLOYER_NAME'] == row['NAME']]
    temp.reset_index()
    for i, r in temp.iterrows():
        c_id = row['COMPANY_ID']
        r_id = roles.loc[roles['NAME'] == r['JOB_TITLE']]
        if len(r_id) == 0:
            continue
        cr = cr.append({'COMPANY_ID': c_id, 'ROLE_ID': r_id['ROLE_ID'].item() }, ignore_index=True)

cr


,COMPANY_ID,ROLE_ID
0,0,2931
1,1,20040
2,2,10749
3,3,13003
4,3,14523
...,...,...
3880,289,14165
3881,289,16124
3882,289,5690
3883,290,16801


In [15]:
for index, row in companies3.iterrows():
    temp = company_roles.loc[company_roles['EMPLOYER_NAME'] == row['NAME']]
    temp.reset_index()
    for i, r in temp.iterrows():
        c_id = row['COMPANY_ID']
        r_id = roles.loc[roles['NAME'] == r['JOB_TITLE']]
        if len(r_id) == 0:
            continue
        cr = cr.append({'COMPANY_ID': c_id, 'ROLE_ID': r_id['ROLE_ID'].item() }, ignore_index=True)

cr

,COMPANY_ID,ROLE_ID
0,0,2931
1,1,20040
2,2,10749
3,3,13003
4,3,14523
...,...,...
4063,524,3569
4064,545,14739
4065,547,17409
4066,552,13294


In [18]:
for index, row in companies.iterrows():
    temp = company_roles2.loc[company_roles2['EMPLOYER_NAME'] == row['NAME']]
    temp.reset_index()
    for i, r in temp.iterrows():
        c_id = row['COMPANY_ID']
        r_id = roles.loc[roles['NAME'] == r['JOB_TITLE']]
        if len(r_id) == 0:
            continue
        cr = cr.append({'COMPANY_ID': c_id, 'ROLE_ID': r_id['ROLE_ID'].item() }, ignore_index=True)

cr

,COMPANY_ID,ROLE_ID
0,0,2931
1,1,20040
2,2,10749
3,3,13003
4,3,14523
...,...,...
5000,285,9324
5001,285,6591
5002,288,19731
5003,289,3708


In [20]:
for index, row in companies3.iterrows():
    temp = company_roles2.loc[company_roles2['EMPLOYER_NAME'] == row['NAME']]
    temp.reset_index()
    for i, r in temp.iterrows():
        c_id = row['COMPANY_ID']
        r_id = roles.loc[roles['NAME'] == r['JOB_TITLE']]
        if len(r_id) == 0:
            continue
        cr = cr.append({'COMPANY_ID': c_id, 'ROLE_ID': r_id['ROLE_ID'].item() }, ignore_index=True)

cr

,COMPANY_ID,ROLE_ID
0,0,2931
1,1,20040
2,2,10749
3,3,13003
4,3,14523
...,...,...
5064,514,15946
5065,519,15946
5066,524,15357
5067,524,16218


In [21]:
cr.to_csv('temp_company_roles.csv')

In [47]:
h1b_comp_roles = pd.read_csv("company_roles.csv", low_memory=False)
h1b_comp_roles.drop(['ROLE_ID'], axis=1, inplace=True)
h1b_comp_roles.rename(columns={"ROLE_NAME": "NAME"}, inplace=True)
h1b_comp_roles

,COMPANY_ROLES_ID,COMPANY_ID,NAME
0,0,0,ASSOCIATE QUANTITATIVE ANALYTICS
1,1,0,ASSOCIATE QUANTITATIVE ANALYTICS
2,2,0,ASSOCIATE QUANTITATIVE ANALYTICS
3,3,0,INVESTMENT ANALYST
4,4,0,ASSOCIATE QUANTITATIVE ANALYTICS
...,...,...,...
64356,64356,556,QUANTITATIVE DATA STRATEGIST
64357,64357,556,PORTFOLIO MANAGER
64358,64358,556,RISK ANALYST
64359,64359,556,ANALYST


In [48]:
new_cr = h1b_comp_roles.merge(roles,on='NAME',how='left')
new_cr = new_cr.astype({'ROLE_ID': 'Int64'})
new_cr.drop(['COMPANY_ROLES_ID', 'NAME'], axis=1, inplace=True)
new_cr

,COMPANY_ID,ROLE_ID
0,0,3695
1,0,3695
2,0,3695
3,0,10695
4,0,3695
...,...,...
64356,556,14364
64357,556,13269
64358,556,14843
64359,556,384


In [49]:
all_comp_roles = new_cr.append(cr)
all_comp_roles = all_comp_roles.drop_duplicates()
all_comp_roles = all_comp_roles.reset_index()
all_comp_roles.drop(['index'], axis=1, inplace=True)
all_comp_roles.index.name = 'COMPANY_ROLES_ID'
all_comp_roles.to_csv('company_roles.csv')